In [5]:
from dash import Dash

app = Dash(__name__, external_stylesheets=external_stylesheets)

if __name__ == '__main__':
    app.run_server(debug=True)

AttributeError: partially initialized module 'dash' has no attribute 'Dash' (most likely due to a circular import)